<a href="https://colab.research.google.com/github/Engineer-Ayesha-Shafique/Brain-Tumor-Segmentation-and-Detection-using-UNET-and-Watershed-in-Python/blob/main/Brain_Tumor_Segmentation_and_Detection_using_UNET_and_Watershed_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive' ,force_remount=True)

Mounted at /content/drive


#UNET

https://www.kaggle.com/datasets/nikhilroxtomar/brain-tumor-segmentation


In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Conv2D,
    BatchNormalization,
    Activation,
    MaxPooling2D,
    Conv2DTranspose,
    Concatenate,
    Input
)

def convolution_block(input_value, no_of_filters):
    """
    A convolutional block consisting of two convolutional layers followed by batch normalization and ReLU activation.

    Args:
        input_value (Tensor): Input tensor.
        no_of_filters (int): Number of filters for convolutional layers.

    Returns:
        Tensor: Output tensor.
    """
    conv_value = Conv2D(no_of_filters,  # Number of filters
                        3,              # Kernel size
                        padding="same"  # Padding to ensure the output has the same spatial dimensions as the input
                        )(input_value)

    conv_value = BatchNormalization()(conv_value)

    conv_value = Activation("relu")(conv_value)

    conv_value = Conv2D(no_of_filters,  # Number of filters
                        3,              # Kernel size
                        padding="same"  # Padding to ensure the output has the same spatial dimensions as the input
                        )(conv_value)

    conv_value = BatchNormalization()(conv_value)

    conv_value = Activation("relu")(conv_value)

    return conv_value

def encoder(input_value, no_of_filters):
    """
    Encoder block that performs a convolutional block followed by max pooling.

    Args:
        input_value (Tensor): Input tensor.
        no_of_filters (int): Number of filters for convolutional layers.

    Returns:
        Tensor: Encoder output tensor.
        Tensor: Pooled output tensor.
    """
    encoder_value = convolution_block(input_value, no_of_filters)
    pool_value = MaxPooling2D((2, 2))(encoder_value)
    return encoder_value, pool_value

def decoder(input_value, skip_features, no_of_filters):
    """
    Decoder block that performs transpose convolution, concatenation with skip connections,
    and a convolutional block.

    Args:
        input_value (Tensor): Input tensor.
        skip_features (Tensor): Skip connection tensor from the encoder block.
        no_of_filters (int): Number of filters for convolutional layers.

    Returns:
        Tensor: Decoder output tensor.
    """
    decoder_value = Conv2DTranspose(no_of_filters, 2, strides=2, padding="same")(input_value)
    decoder_value = Concatenate()([decoder_value, skip_features])
    decoder_value = convolution_block(decoder_value, no_of_filters)
    return decoder_value

def Unet_model(input_shape):
    """
    UNet model implementation with an input shape.

    Args:
        input_shape (tuple): Shape of the input data.

    Returns:
        Model: UNet model.
    """
    # Input layer
    input_value = Input(input_shape)

    # Encoder
    encoder_1_value, pool_1_value = encoder(input_value, 64)
    encoder_2_value, pool_2_value = encoder(pool_1_value, 128)
    encoder_3_value, pool_3_value = encoder(pool_2_value, 256)
    encoder_4_value, pool_4_value = encoder(pool_3_value, 512)

    # Convolution block
    conv_value_1 = convolution_block(pool_4_value, 1024)

    # Decoder
    decoder_1_value = decoder(conv_value_1, encoder_4_value, 512)
    decoder_2_value = decoder(decoder_1_value, encoder_3_value, 256)
    decoder_3_value = decoder(decoder_2_value, encoder_2_value, 128)
    decoder_4_value = decoder(decoder_3_value, encoder_1_value, 64)

    #Output Layer
    # ✅ New (multi-class segmentation: 4 classes)
    output_value = Conv2D(4, 1, padding="same", activation="softmax")(decoder_4_value)


    #Model compliation
    model_result = Model(input_value,          # Input layer
                  output_value,         # Output layer
                  name="UNET"           # Model name (optional)
    )

    return  model_result

if __name__ == "__main__":

    # Defines the shape of the input data: 256x256 pixels with 3 color channels (RGB)
    input_shape = (224, 224, 3)

    # Creates an instance of the UNet model with the specified input shape
    model_result = Unet_model(input_shape)

    # Prints a summary of the model's architecture, including the number of parameters and layer shapes
    model_result.summary()


Model: "UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 224, 224,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 224, 224,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_4[0][0]  

 Total params: 31,055,492 (118.47 MB)

 Trainable params: 31,043,716 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

# Metrics

In [ ]:
import tensorflow as tf

smooth = 1e-15
num_classes = 5  # <-- make this match your final Conv2D(filters=5, ...)

def dice_coefficient(y_true, y_pred):
    # y_true: (B, H, W, 1) integers 0..4
    # y_pred: (B, H, W, 5) softmax probs

    # 1) Squeeze off the last dim → (B, H, W)
    y_true = tf.squeeze(y_true, axis=-1)

    # 2) One‑hot encode → (B, H, W, 5)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)

    # 3) Flatten both to (B*H*W, 5)
    y_true_f = tf.reshape(y_true_o, [-1, num_classes])
    y_pred_f = tf.reshape(y_pred,   [-1, num_classes])

    # 4) Compute intersection & union per class
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union        = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)

    # 5) Dice per class, then average
    dice_per_class = (2. * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)


In [ ]:
import cv2
import os
import numpy as np

mask_dir = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"

for fname in os.listdir(mask_dir):
    if fname.endswith('.tif'):
        path = os.path.join(mask_dir, fname)
        mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        if mask is None:
            print(f"[ERROR] Cannot read: {fname}")
            continue
        unique_vals = np.unique(mask)
        if not np.all(np.isin(unique_vals, [0, 1, 2, 3])):
            print(f"[WARNING] {fname} contains invalid labels: {unique_vals}")


[WARNING] VeryMildDemented_1e80236e-2242-459c-938e-fb37a51bb0a5_jpg.rf.04eb4dca9ac60ab8397a2f5054252cce_mask.tif contains invalid labels: [0 4]
[WARNING] VeryMildDemented_1e80236e-2242-459c-938e-fb37a51bb0a5_jpg.rf.27b5712e98714cec52d9643ccf1da710_mask.tif contains invalid labels: [0 4]
[WARNING] VeryMildDemented_1ed676e7-9737-4f0a-92cb-a599dd82e015_jpg.rf.ae505ddd5245c63ea9591a2520e4f113_mask.tif contains invalid labels: [0 4]
[WARNING] VeryMildDemented_1efc232b-f83d-44e0-b113-d8a56e4ef9c9_jpg.rf.3c983ebe178ea17806f8c31ebadc6db9_mask.tif contains invalid labels: [0 4]
[WARNING] VeryMildDemented_1f43cf97-f8c1-4564-9b55-8d7f25ec41f0_jpg.rf.c8dc7ad10df630c2514310463f5e2a39_mask.tif contains invalid labels: [0 4]
[WARNING] VeryMildDemented_1f6e146a-4064-4e5a-af95-34f21282f2df_jpg.rf.60b003518b592e306c852e346c8e7755_mask.tif contains invalid labels: [0 4]
[WARNING] VeryMildDemented_1feb78ff-c629-4bd4-8f10-cd21385ab3ff_jpg.rf.c28647bc9fcf960b25523be916594873_mask.tif contains invalid labels

KeyboardInterrupt: 

#Train


In [ ]:
import os

def add_mask_suffix_to_tif(dir_path):
    """
    Rename all .tif files in dir_path by adding '_mask' before the .tif extension.

    E.g., 'image1.tif' → 'image1_mask.tif'
    """
    for fname in os.listdir(dir_path):
        if fname.lower().endswith('.tif'):
            old_path = os.path.join(dir_path, fname)
            base = fname[:-4]  # strip off ".tif"
            new_name = f"{base}_mask.tif"
            new_path = os.path.join(dir_path, new_name)
            os.rename(old_path, new_path)
    print(f"Done renaming all .tif files in {dir_path}")

add_mask_suffix_to_tif('/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks')
add_mask_suffix_to_tif('/content/drive/MyDrive/Project_41725/newdata/val_maskimg/masks')

KeyboardInterrupt: 

In [9]:
import os
import re

mask_dir = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"

for fname in sorted(os.listdir(mask_dir)):
    if not fname.lower().endswith('.tif'):
        continue

    stem, ext = os.path.splitext(fname)

    if re.search(r'_mask(?:_mask)*$', stem):
        # collapse multiple "_mask" → exactly one
        new_stem = re.sub(r'(?:_mask)+$', '_mask', stem)
    else:
        # no "_mask" suffix at all → add one
        new_stem = stem + '_mask'

    if new_stem != stem:
        new_name = new_stem + ext
        os.rename(
            os.path.join(mask_dir, fname),
            os.path.join(mask_dir, new_name)
        )
        print(f"RENAMED: {fname} → {new_name}")

print("Done.")


RENAMED: MildDemented_00be34e4-c61c-45a8-8ee8-48e29a8adce0_jpg.rf.bc8e786abf4ec687946d6ebbbe11300e_mask_mask.tif → MildDemented_00be34e4-c61c-45a8-8ee8-48e29a8adce0_jpg.rf.bc8e786abf4ec687946d6ebbbe11300e_mask.tif
RENAMED: MildDemented_01bb5650-386e-4bdd-854e-54e12030a3c0_jpg.rf.91288ed2e7955c1255997e31afe67b6f_mask_mask.tif → MildDemented_01bb5650-386e-4bdd-854e-54e12030a3c0_jpg.rf.91288ed2e7955c1255997e31afe67b6f_mask.tif
RENAMED: MildDemented_01dc8d82-81d9-4344-a80e-cf981232df36_jpg.rf.f1975ff08e1b64dba4100aafd50aa3aa_mask_mask.tif → MildDemented_01dc8d82-81d9-4344-a80e-cf981232df36_jpg.rf.f1975ff08e1b64dba4100aafd50aa3aa_mask.tif
RENAMED: MildDemented_02addcbc-619b-4a93-bca0-fb21e0ab7066_jpg.rf.5b10a3fa0c4f10c0b6cf689bf9766e1c_mask_mask.tif → MildDemented_02addcbc-619b-4a93-bca0-fb21e0ab7066_jpg.rf.5b10a3fa0c4f10c0b6cf689bf9766e1c_mask.tif
RENAMED: MildDemented_02f04a0c-fd24-4562-b1a6-647312898f1d_jpg.rf.e6db83154a7231dba56178dcc7cf2f2e_mask_mask.tif → MildDemented_02f04a0c-fd24-45

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 1) Import TF first and set memory growth (fallback to experimental)
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except Exception as e:
            print(f"[GPU] Could not set memory growth: {e}")
    print(f"[GPU] Detected {len(gpus)} GPU(s).")

# 2) Now import the rest
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import (
    ModelCheckpoint, CSVLogger, ReduceLROnPlateau,
    EarlyStopping, TensorBoard
)
from tensorflow.keras.optimizers import Adam

""" Global parameters """
Height = 224
Width  = 224

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path_dataset, val_ratio=0.2, test_ratio=0.1, random_state=42):
    img_dir  = os.path.join(path_dataset, "images")
    mask_dir = os.path.join(path_dataset, "masks")

    image_exts = ('.png', '.jpg', '.jpeg')
    images = sorted([
        os.path.join(img_dir, f)
        for f in os.listdir(img_dir)
        if f.lower().endswith(image_exts)
    ])


    mask_exts = ('.tif', '.tiff')
    masks = sorted([
        os.path.join(mask_dir, f)
        for f in os.listdir(mask_dir)
        if f.lower().endswith(mask_exts)
    ])

    if not images:
        raise RuntimeError(f"No images found in {img_dir}")
    if not masks:
        raise RuntimeError(f"No masks found in {mask_dir}")

    images_lookup = {
        os.path.splitext(os.path.basename(p))[0]: p
        for p in images
    }
#MildDemented_0a7b1321-eba0-40dc-85b8-de34241554a2_jpg.rf.70370fcd9ac0fb82f91d27f558053d84.tif
    pairs = []
    for m in masks:
        base = os.path.splitext(os.path.basename(m))[0]
        if base.endswith('_mask'):
            root = base[:-5]
            img_path = images_lookup.get(root)
            if img_path:
                pairs.append((img_path, m))

    if not pairs:
        raise RuntimeError("No matching image↔mask pairs found!")

    trainval, test = train_test_split(pairs, test_size=test_ratio, random_state=random_state)
    val_frac = val_ratio / (1.0 - test_ratio)
    train, val = train_test_split(trainval, test_size=val_frac, random_state=random_state)

    train_imgs, train_masks = zip(*train)
    val_imgs,   val_masks   = zip(*val)
    test_imgs,  test_masks  = zip(*test)

    return (
        list(train_imgs), list(train_masks),
        list(val_imgs),   list(val_masks),
        list(test_imgs),  list(test_masks)
    )

def read_image_file(path_img):
    path = path_img.decode('utf-8') if isinstance(path_img, bytes) else path_img
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height)) / 255.0
    return img.astype(np.float32)

def read_mask_file(path_mask):
    path = path_mask.decode('utf-8') if isinstance(path_mask, bytes) else path_mask
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask.ndim == 3:
        mask = mask[..., 0]
    mask = cv2.resize(mask, (Width, Height), interpolation=cv2.INTER_NEAREST)

    # ✅ Clip to [0, 4], not [0, 3]:
    mask = np.clip(mask, 0, NUM_CLASSES-1)

    # Convert to integer class IDs
    mask = mask.astype(np.int32)

    # Add the channel dim back
    return mask[..., None]

def tf_parse(path_img, path_mask):
    # Tell numpy_function: img→float32, mask→int32
    img, mask = tf.numpy_function(
        lambda pi, pm: (read_image_file(pi), read_mask_file(pm)),
        [path_img, path_mask],
        [tf.float32, tf.int32]          # <-- second output is now int32
    )
    img.set_shape([Height, Width, 3])
    mask.set_shape([Height, Width, 1])
    return img, mask


def tf_dataset(inputs, targets, batch_size=2):
    ds = tf.data.Dataset.from_tensor_slices((inputs, targets))
    ds = ds.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

def edit_mask_images(mask_dir, func, output_dir=None):
    out_dir = output_dir or mask_dir
    create_folder(out_dir)
    exts = ('.tif')
    for fname in os.listdir(mask_dir):
        if not fname.lower().endswith(exts):
            continue
        in_path  = os.path.join(mask_dir, fname)
        mask     = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)
        edited   = func(mask)
        cv2.imwrite(os.path.join(out_dir, fname), edited)
    print(f"[edit_mask_images] Applied edits to masks in {out_dir}")

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_folder("/content/drive/MyDrive/Project_41725/files")

    batch_size = 16
    lr         = 1e-4
    epochs_num     = 100
    path_model = os.path.join("/content/drive/MyDrive/Project_41725/files", "model.h5")
    path_csv   = os.path.join("/content/drive/MyDrive/Project_41725/files", "log.csv")

    ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(ds_path)

    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

    train_ds = tf_dataset(train_imgs, train_masks, batch_size=batch_size)
    val_ds   = tf_dataset(val_imgs,   val_masks,   batch_size=batch_size)
    test_ds  = tf_dataset(test_imgs,  test_masks,  batch_size=batch_size)

    # Build, compile and fit your model below:
    # model = Unet_model((Height, Width, 3))
    # model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coeff])
    # callbacks = [ModelCheckpoint(...), ...]
    # model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)


[GPU] Could not set memory growth: Physical devices cannot be modified after being initialized
[GPU] Detected 1 GPU(s).
Train: 24, Val: 7, Test: 4


In [ ]:
for msk in train_masks:
    arr = cv2.imread(msk, cv2.IMREAD_UNCHANGED)
    if arr is None: continue
    if np.any(arr > 3):
        print("[INVALID]", msk, np.unique(arr))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
)

NUM_CLASSES = 5
INPUT_SHAPE = (224, 224, 3)
EPOCHS      = 160

# 1) Build your base U‑Net (this returns a model ending in a single-channel output)
base = Unet_model(INPUT_SHAPE)

# 2) Replace its final layer with a 5‑channel softmax
#    We assume the penultimate layer is base.layers[-2]
x = base.layers[-2].output
outputs = Conv2D(
    NUM_CLASSES,
    kernel_size=(1,1),
    activation='softmax',
    name='multiclass_output'
)(x)

# 3) Create the new Model
model = Model(inputs=base.input, outputs=outputs)

# 4) Loss, metrics, and compile (as before)
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=NUM_CLASSES)
    y_true_f = tf.reshape(y_true_o, [-1, NUM_CLASSES])
    y_pred_f = tf.reshape(y_pred,   [-1, NUM_CLASSES])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    denom = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice_per_class = (2. * intersection + smooth) / (denom + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)(y_true, y_pred)
    return ce + dice_loss(y_true, y_pred)

class MeanIoUArgMax(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes=NUM_CLASSES, name="mean_iou", dtype=None):
        super().__init__(num_classes=num_classes, name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

model.compile(
    optimizer=Adam(1e-4),
    loss=combined_loss,
    metrics=[dice_coef, MeanIoUArgMax()]
)

# 5) (Optional) Evaluate before training
print("Before training:", dict(zip(model.metrics_names, model.evaluate(test_ds))))

# 6) Callbacks
callbacks = [
    ModelCheckpoint(path_model, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(path_csv),
    EarlyStopping(monitor='val_loss', patience=20),
]

# 7) Train
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - dice_coef: 0.1108 - loss: 2.5038 - mean_iou: 0.0550
Before training: {'loss': 2.5037879943847656, 'compile_metrics': 0.11082456260919571}
Epoch 1/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - dice_coef: 0.1245 - loss: 2.4548 - mean_iou: 0.0660 
Epoch 1: val_loss improved from inf to 2.49320, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 77s 40s/step - dice_coef: 0.1278 - loss: 2.4343 - mean_iou: 0.0701 - val_dice_coef: 0.1117 - val_loss: 2.4932 - val_mean_iou: 0.0402 - learning_rate: 1.0000e-04
Epoch 2/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - dice_coef: 0.1966 - loss: 1.8678 - mean_iou: 0.2236 
Epoch 2: val_loss improved from 2.49320 to 2.48326, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - dice_coef: 0.2001 - loss: 1.8528 - mean_iou: 0.2281 - val_dice_coef: 0.1125 - val_loss: 2.4833 - val_mean_iou: 0.1190 - learning_rate: 1.0000e-04
Epoch 3/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - dice_coef: 0.2461 - loss: 1.5664 - mean_iou: 0.2934
Epoch 3: val_loss improved from 2.48326 to 2.47360, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - dice_coef: 0.2491 - loss: 1.5537 - mean_iou: 0.2961 - val_dice_coef: 0.1132 - val_loss: 2.4736 - val_mean_iou: 0.1333 - learning_rate: 1.0000e-04
Epoch 4/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - dice_coef: 0.2761 - loss: 1.3937 - mean_iou: 0.3167
Epoch 4: val_loss improved from 2.47360 to 2.46378, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - dice_coef: 0.2787 - loss: 1.3817 - mean_iou: 0.3190 - val_dice_coef: 0.1139 - val_loss: 2.4638 - val_mean_iou: 0.1340 - learning_rate: 1.0000e-04
Epoch 5/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - dice_coef: 0.2952 - loss: 1.2847 - mean_iou: 0.3282
Epoch 5: val_loss improved from 2.46378 to 2.45329, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - dice_coef: 0.3847 - loss: 0.6873 - mean_iou: 0.9956 - val_dice_coef: 0.1980 - val_loss: 1.8892 - val_mean_iou: 0.5122 - learning_rate: 1.0000e-04
Epoch 56/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - dice_coef: 0.3847 - loss: 0.6871 - mean_iou: 0.9951
Epoch 56: val_loss improved from 1.88916 to 1.88486, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - dice_coef: 0.3849 - loss: 0.6863 - mean_iou: 0.9952 - val_dice_coef: 0.1994 - val_loss: 1.8849 - val_mean_iou: 0.5099 - learning_rate: 1.0000e-04
Epoch 57/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - dice_coef: 0.3848 - loss: 0.6863 - mean_iou: 0.9955 
Epoch 57: val_loss improved from 1.88486 to 1.87946, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - dice_coef: 0.3850 - loss: 0.6854 - mean_iou: 0.9956 - val_dice_coef: 0.2009 - val_loss: 1.8795 - val_mean_iou: 0.5087 - learning_rate: 1.0000e-04
Epoch 58/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - dice_coef: 0.3850 - loss: 0.6851 - mean_iou: 0.9953
Epoch 58: val_loss improved from 1.87946 to 1.87681, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - dice_coef: 0.3852 - loss: 0.6843 - mean_iou: 0.9954 - val_dice_coef: 0.2022 - val_loss: 1.8768 - val_mean_iou: 0.5061 - learning_rate: 1.0000e-04
Epoch 59/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - dice_coef: 0.3852 - loss: 0.6841 - mean_iou: 0.9960
Epoch 59: val_loss did not improve from 1.87681
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - dice_coef: 0.3854 - loss: 0.6833 - mean_iou: 0.9961 - val_dice_coef: 0.2032 - val_loss: 1.8772 - val_mean_iou: 0.5022 - learning_rate: 1.0000e-04
Epoch 60/160
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - dice_coef: 0.3847 - loss: 0.6856 - mean_iou: 0.9955
Epoch 60: val_loss improved from 1.87681 to 1.87364, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - dice_coef: 0.3855 - loss: 0.6823 - mean_iou: 0.9958 - val_dice_coef: 0.2046 - val_loss: 1.8736 - val_mean_iou: 0.5009 - learning_rate: 1.0000e-04
Epoch 61/160
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - dice_coef: 0.3855 - loss: 0.6822 - mean_iou: 0.9962
Epoch 61: val_loss did not improve from 1.87364
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - dice_coef: 0.3857 - loss: 0.6814 - mean_iou: 0.9963 - val_dice_coef: 0.2054 - val_loss: 1.8753 - val_mean_iou: 0.4972 - learning_rate: 1.0000e-04
Epoch 62/160
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - dice_coef: 0.3851 - loss: 0.6835 - mean_iou: 0.9965
Epoch 62: val_loss did not improve from 1.87364
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - dice_coef: 0.3859 - loss: 0.6804 - mean_iou: 0.9967 - val_dice_coef: 0.2066 - val_loss: 1.8739 - val_mean_iou: 0.4954 - learning_rate: 1.0000e-04
Epoch 63/160
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - dice_coef: 0.3852 - loss: 0.6828 - mean_iou: 0.9955
Epoch 63: val_loss did not im

#Test


In [ ]:
from google.colab import drive
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# 1) Mount drive
drive.mount('/content/drive', force_remount=True)

# 2) Make sure your dice functions are in scope if you ever re‑compile
# def dice_coefficient(...): ...
# def dice_coefficient_loss(...): ...

model_path = "/content/drive/MyDrive/Project_41725/files/model.h5"

# 3) Load model for inference only—skip compile to avoid the TypeError
model = load_model(
    model_path,
    custom_objects={
        'dice_coefficient': dice_coefficient,
        'dice_coefficient_loss': dice_coefficient_loss
    },
    compile=False
)

def predict_and_show(img_path, model, input_size=(224,224)):
    gray    = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    resized = cv2.resize(gray, input_size)
    x       = np.stack([resized]*3, -1).astype(np.float32) / 255.0

    # Run model
    pred = model.predict(x[None])[0]         # ➔ (H, W, num_classes)
    seg  = pred.argmax(-1).astype(np.uint8)  # ➔ (H, W)

    # Compute and print class percentages
    unique, counts = np.unique(seg, return_counts=True)
    total = seg.size
    percentages = {int(cls): float(cnt/total*100) for cls, cnt in zip(unique, counts)}
    print(f"\nClass percentages for {img_path}:")
    for cls, pct in percentages.items():
        print(f"  Class {cls}: {pct:.2f}%")

    # Display images
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.title("Input")
    plt.imshow(resized, cmap='gray')
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.title("Segmentation Overlay")
    plt.imshow(resized, cmap='gray', alpha=0.5)
    plt.imshow(seg, cmap='jet',   alpha=0.5)
    plt.axis('off')
    plt.show()

# 4) Upload & run on uploaded files
uploaded = files.upload()
for fn in uploaded:
    predict_and_show(fn, model)
    print("Num output channels:", model.output_shape[-1])


In [ ]:
import numpy as np

counts = np.zeros(4, dtype=int)
for mpath in train_masks:
    mask = cv2.imread(mpath, cv2.IMREAD_GRAYSCALE).flatten()
    for c in range(4):
        counts[c] += np.sum(mask == c)

freqs = counts / counts.sum()
print("Class frequencies:", {i: freqs[i] for i in range(4)})


In [ ]:
import tifffile
import numpy as np

for mpath in train_masks[:2000]:
    mask = tifffile.imread(mpath)            # read exact IDs
    print(mpath, np.unique(mask))


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

""" Global parameters """
Height = 224
Width = 224

""" Creating a directory in drive """
def create_folder(path):
    """
    Create a directory if it does not exist.

    Args:
        path: Path to the directory.
    """
    if not os.path.exists(path):
        os.makedirs(path)

def Result_saving(inp_image, m_true, p_mask, path_saved_image):
    """
    Save the image, ground truth mask, and predicted mask side by side.

    Args:
        inp_image: Input image.
        m_true: Ground truth mask.
        p_mask: Predicted mask.
        path_saved_image: Path to save the image.
    """
    m_true  =  np.expand_dims ( m_true, axis=-1)
    m_true  =  np.concatenate ( [ m_true, m_true, m_true], axis=-1 )

    p_mask = np.expand_dims ( p_mask, axis=-1)
    p_mask = np.concatenate ( [ p_mask, p_mask, p_mask], axis=-1)
    p_mask = p_mask * 255

    line = np.ones((Height, 10, 3)) * 255

    concat_images = np.concatenate([inp_image, line, m_true, line, p_mask], axis=1)
    cv2.imwrite(path_saved_image, concat_images )

if __name__ == "__main__":
    """ Seeding the tensor """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_folder("/content/drive/MyDrive/DSP Project/results")

    from tensorflow.keras.models import load_model

# Define custom metrics and loss
    def dice_coefficient(y_true, y_pred, smooth=1e-6):
      y_true_f = tf.keras.backend.flatten(y_true)
      y_pred_f = tf.keras.backend.flatten(y_pred)
      intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
      return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

    def dice_coefficient_loss(y_true, y_pred):
      return 1.0 - dice_coefficient(y_true, y_pred)

    # Define this dictionary BEFORE calling load_model
    custom_objects = {
         "dice_coefficient": dice_coefficient,
         "dice_coefficient_loss": dice_coefficient_loss
      }

    # Now safely load the model
    model_loaded = load_model("/content/drive/MyDrive/Project_41725/files/model.h5", custom_objects=custom_objects)

    """ loading the dataset from drive"""
    path_dataset ="/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(path_dataset)


    """ Model Prediction and Evaluation along with score saving to drive """
    EVL_SCORE_list = []
    for im, msk in tqdm(zip(test_imgs, test_masks), total=len(test_masks)):
        """ Extracting the name of image """
        n_img = im.split("/")[-1]

        """ Reading the image """
        inp_image = cv2.imread(im, cv2.IMREAD_COLOR) ## [Height, Width, 3]
        inp_image = cv2.resize(inp_image, (Width, Height))       ## [Height, Width, 3]
        im = inp_image/255.0                         ## [Height, Width, 3]
        im = np.expand_dims(im, axis=0)           ## [1, Height, Width, 3]

        """ Reading the mask """
        m_true = cv2.imread(msk, cv2.IMREAD_GRAYSCALE)
        m_true = cv2.resize(m_true, (Width, Height))

        """ Predicting mask """
        p_mask = model_loaded.predict(im, verbose=0)[0]              # Output shape: (224, 224, 4)
        p_mask = np.argmax(p_mask, axis=-1).astype(np.uint8)         # Convert softmax to class ID


        """ Save the predicting mask result """
        path_saved_image = os.path.join("/content/drive/MyDrive/DSP Project/results", n_img)
        Result_saving(inp_image, m_true, p_mask, path_saved_image)

        """ Flatten the array """
        m_true = m_true.astype(np.uint8).flatten()
        p_mask = p_mask.flatten()

        """ finding the evalution value """
        f1_scoree        = f1_score(m_true, p_mask.flatten(), average="macro")
        jaccard_scoree   = jaccard_score(m_true, p_mask.flatten(), average="macro")
        recall_scoree    = recall_score(m_true, p_mask.flatten(), average="macro", zero_division=0)
        precision_scoree = precision_score(m_true, p_mask.flatten(), average="macro", zero_division=0)


    """ EEvluation Matrix """
    evl_score_val = [s[1:]for s in EVL_SCORE_list]
    evl_score_val = np.mean(evl_score_val, axis=0)
    print(f"F1: {evl_score_val[0]:0.5f}")
    print(f"Jaccard: {evl_score_val[1]:0.5f}")
    print(f"Recall: {evl_score_val[2]:0.5f}")
    print(f"Precision: {evl_score_val[3]:0.5f}")

    df = pd.DataFrame(EVL_SCORE_list, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("/content/drive/MyDrive/DSP Project/files/score.csv")


#Plot and Checks

In [ ]:
import os
import shutil

def move_images(src_folder, dst_folder):
    """
    Move all image files from src_folder to dst_folder.

    Args:
        src_folder (str): Source directory containing image files.
        dst_folder (str): Destination directory where images will be moved.
    """
    os.makedirs(dst_folder, exist_ok=True)
    image_exts = ('.png', '.jpg', '.jpeg')

    moved = 0
    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith(image_exts):
            src_path = os.path.join(src_folder, file_name)
            dst_path = os.path.join(dst_folder, file_name)
            shutil.move(src_path, dst_path)
            moved += 1

    print(f"[INFO] Moved {moved} image(s) from {src_folder} → {dst_folder}")

move_images("/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks", "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/ignore")


In [ ]:
import pandas as pd
metrics = pd.read_csv("/content/drive/MyDrive/DSP Project/files/log.csv")
metrics.head(10)

In [ ]:
metrics[['dice_coef','val_dice_coef']].plot()

In [ ]:
metrics[['loss','val_loss']].plot()

In [ ]:
scores = pd.read_csv("/content/drive/MyDrive/DSP Project/files/score.csv")
scores.head(10)

#Watershed implementation for automataing segmentation

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.segmentation import watershed as skwater
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

def ShowImage(list_img, list_t, list_cm):
    """
    Display multiple images with corresponding titles and colormaps.

    Args:
        list_img (list): List of images to be displayed.
        list_t (list): List of titles for each image.
        list_cm (list): List of colormaps for each image.
    """
    no_of_img = len(list_img)
    figure, axis_label = plt.subplots(1, no_of_img, figsize = (10*no_of_img, 10))
    for x  in range(no_of_img):
        axis_label[x].imshow(list_img[x], cmap=list_cm[x])
        axis_label[x].axis('off')
        axis_label[x].set_title(list_t[x])
    plt.show()

def watershed(path_img):
    """
    Apply watershed segmentation to an image.

    Args:
        path_img (str): Path to the input image.
    """
    # Read the image
    color_image = cv2.imread(path_img)
    grey_img = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to obtain a binary image
    computed_threshold, threshold = cv2.threshold(grey_img, 0, 255, cv2.THRESH_OTSU)
    computed_threshold, labeled_image = cv2.connectedComponents(threshold)

    # Find the largest component (presumed to be the brain)
    marker_area = [np.sum(labeled_image == m) for m in range(1, np.max(labeled_image))]
    largest_component = np.argmax(marker_area) + 1
    foreground = labeled_image == largest_component
    brain_out = color_image.copy()
    brain_out[foreground == False] = (0, 0, 0)

    # Apply morphological operations to refine the segmentation
    grey_img = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    computed_threshold, threshold = cv2.threshold(grey_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel, iterations=2)
    background = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    computed_threshold, transform_threshold = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    transform_threshold = np.uint8(transform_threshold)
    unknown = cv2.subtract(background, transform_threshold)
    computed_threshold, labeled_image = cv2.connectedComponents(transform_threshold)
    labeled_image = labeled_image + 1
    labeled_image[unknown == 255] = 0

    # Perform watershed segmentation
    labeled_image = cv2.watershed(color_image, labeled_image)
    color_image[labeled_image == -1] = [255, 0, 0]
    im1 = cv2.cvtColor(color_image, cv2.COLOR_HSV2RGB)

    # Apply morphological closing to refine the brain mask
    foreground = np.uint8(foreground)
    kernel = np.ones((8, 8), np.uint8)
    closing = cv2.morphologyEx(foreground, cv2.MORPH_CLOSE, kernel)

    brain_out = color_image.copy()
    brain_out[closing == False] = (0, 0, 0)

    # Prepare images, titles, and colormaps for visualization
    images = [grey_img, threshold, im1, closing]
    titles = ['Brain MRI', 'Thresholding image', 'Watershed segmented image', 'Closing']
    cmaps = ['gray', 'gray', 'gray', 'gray']

    # Display the images
    ShowImage(images, titles, cmaps)


#AI VS DSP

In [ ]:
import matplotlib.pyplot as plt


dir = "/content/drive/MyDrive/DSP Project/results/"
images = os.listdir("/content/drive/MyDrive/DSP Project/results/")[:15]
a  = "33.png"
img1 = plt.imread(dir+ a)
plt.imshow(img1, cmap='gray')

img_path = "/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a
print(watershed("/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a))

In [ ]:
import matplotlib.pyplot as plt


dir = "/content/drive/MyDrive/DSP Project/results/"
images = os.listdir("/content/drive/MyDrive/DSP Project/results/")[:15]
a  = images[2]
img1 = plt.imread(dir+ a)
plt.imshow(img1, cmap='gray')

img_path = "/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a
print(watershed("/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a))

In [ ]:
import matplotlib.pyplot as plt


dir = "/content/drive/MyDrive/DSP Project/results/"
images = os.listdir("/content/drive/MyDrive/DSP Project/results/")[:15]
a  = images[10]
img1 = plt.imread(dir+ a)
plt.imshow(img1, cmap='gray')

img_path = "/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a
print(watershed("/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a))